## Yolov5 high resolution training

### Major modification
* img=3600
* mixup=0.5
* fliplr: 0.5

### Hardware to reproduce
* RTX3090

In [1]:
#https://www.kaggle.com/sentrankim/fast-augmentation-by-albumentation-multi-process

# image Enhancement

#### image enhancment end

# Train

In [2]:
import wandb
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()

# I have saved my API token with "wandb_api" as Label. 
# If you use some other Label make sure to change the same below. 
wandb_api = '7f0b0b26236eae67c25381f94363123b539faad3'
#user_secrets.get_secret("wandb_api") 

wandb.login(key=wandb_api)

wandb: W&B API key is configured (use `wandb login --relogin` to force relogin)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publically.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:
from wandb.keras import WandbCallback

wandb.login()

wandb: Currently logged in as: mochi22 (use `wandb login --relogin` to force relogin)


True

In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from shutil import copyfile
import os
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"]= "true"
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

In [5]:
import tensorflow_addons as tfa
import warnings
warnings.filterwarnings('ignore')
import cv2
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt
import seaborn as sns
import random
#import ffmpeg
#from IPython.display import Video
from tqdm import tqdm
import tensorflow as tf
import tensorflow_addons as tfa
import logging
from itertools import cycle

import cv2
import numpy as np

logging.disable(logging.WARNING)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 


plt.style.use('ggplot')
cm = sns.light_palette("green", as_cmap=True)
pd.option_context('display.max_colwidth', 100)
color_pal = plt.rcParams["axes.prop_cycle"].by_key()["color"]
color_cycle = cycle(plt.rcParams["axes.prop_cycle"].by_key()["color"])
# SEED EVERYTHING
random.seed(hash("setting random seeds") % 2**32 - 1)
np.random.seed(hash("improves reproducibility") % 2**32 - 1)

In [6]:
#train = pd.read_csv('../input/tensorflow-great-barrier-reef/train.csv')
train = pd.read_csv('../input/reef-cv-strategy-subsequences-dataframes/cross-validation/train-5folds.csv')
train['pos'] = train.annotations != '[]'
train

,video_id,sequence,video_frame,sequence_frame,image_id,annotations,n_annotations,has_annotations,image_path,subsequence_id,fold,pos
0,0,40258,0,0,0-0,[],0,False,../input/tensorflow-great-barrier-reef/train_i...,1,2,False
1,0,40258,1,1,0-1,[],0,False,../input/tensorflow-great-barrier-reef/train_i...,1,2,False
2,0,40258,2,2,0-2,[],0,False,../input/tensorflow-great-barrier-reef/train_i...,1,2,False
3,0,40258,3,3,0-3,[],0,False,../input/tensorflow-great-barrier-reef/train_i...,1,2,False
4,0,40258,4,4,0-4,[],0,False,../input/tensorflow-great-barrier-reef/train_i...,1,2,False
...,...,...,...,...,...,...,...,...,...,...,...,...
23496,2,29859,10755,2983,2-10755,[],0,False,../input/tensorflow-great-barrier-reef/train_i...,137,3,False
23497,2,29859,10756,2984,2-10756,[],0,False,../input/tensorflow-great-barrier-reef/train_i...,137,3,False
23498,2,29859,10757,2985,2-10757,[],0,False,../input/tensorflow-great-barrier-reef/train_i...,137,3,False
23499,2,29859,10758,2986,2-10758,[],0,False,../input/tensorflow-great-barrier-reef/train_i...,137,3,False


In [7]:
df4 = pd.read_csv('../input/cropdataframes/cropdf_0.4')
df4 = df4[['alls_4','path_4','nonpass4']]
df4

,alls_4,path_4,nonpass4
0,"{'x': 256, 'y': 144, 'width': 100, 'height': 64}",./img512_288/cots_crops/cotscrop-video_0-16-0.jpg,./img512_288/notcots_crops/notcotscrop-video_2...
1,"{'x': 256, 'y': 144, 'width': 100, 'height': 64}",./img512_288/cots_crops/cotscrop-video_0-17-0.jpg,./img512_288/notcots_crops/notcotscrop-video_1...
2,"{'x': 256, 'y': 144, 'width': 100, 'height': 64}",./img512_288/cots_crops/cotscrop-video_0-18-0.jpg,./img512_288/notcots_crops/notcotscrop-video_2...
3,"{'x': 256, 'y': 144, 'width': 100, 'height': 64}",./img512_288/cots_crops/cotscrop-video_0-19-0.jpg,./img512_288/notcots_crops/notcotscrop-video_0...
4,"{'x': 256, 'y': 144, 'width': 100, 'height': 64}",./img512_288/cots_crops/cotscrop-video_0-20-0.jpg,./img512_288/notcots_crops/notcotscrop-video_1...
...,...,...,...
11893,"{'x': 256, 'y': 144, 'width': 80, 'height': 74}",./img512_288/cots_crops/cotscrop-video_2-10628...,./img512_288/notcots_crops/notcotscrop-video_1...
11894,"{'x': 256, 'y': 144, 'width': 80, 'height': 74}",./img512_288/cots_crops/cotscrop-video_2-10629...,./img512_288/notcots_crops/notcotscrop-video_0...
11895,"{'x': 256, 'y': 144, 'width': 82, 'height': 74}",./img512_288/cots_crops/cotscrop-video_2-10630...,./img512_288/notcots_crops/notcotscrop-video_1...
11896,"{'x': 256, 'y': 144, 'width': 88, 'height': 74}",./img512_288/cots_crops/cotscrop-video_2-10631...,./img512_288/notcots_crops/notcotscrop-video_1...


In [8]:
import ast
areas=[]
areas_1000 = []
for i in df4['alls_4']:
    i = ast.literal_eval(i)
    w = i['width']
    h = i['height']
    area = w*h
    area_1000 = int(area/1000)
    areas.append(area)
    areas_1000.append(area_1000)
print(len(areas),len(areas_1000))

11898 11898


In [9]:
path = []
for i in df4['path_4']:
    path.append(i[13:])
print(len(path),path[:5])

11898 ['cots_crops/cotscrop-video_0-16-0.jpg', 'cots_crops/cotscrop-video_0-17-0.jpg', 'cots_crops/cotscrop-video_0-18-0.jpg', 'cots_crops/cotscrop-video_0-19-0.jpg', 'cots_crops/cotscrop-video_0-20-0.jpg']


In [10]:
nonpath = []
for i in df4['nonpass4']:
    nonpath.append(i[13:])


In [11]:
print(len(nonpath),nonpath[:5])

11898 ['notcots_crops/notcotscrop-video_2-4331-0.jpg', 'notcots_crops/notcotscrop-video_1-8615-0.jpg', 'notcots_crops/notcotscrop-video_2-513-0.jpg', 'notcots_crops/notcotscrop-video_0-2020-0.jpg', 'notcots_crops/notcotscrop-video_1-819-0.jpg']


In [12]:
df4['path'] = path
df4['nonpath'] = nonpath

In [13]:
zeros = [0]*11898
df4['nonalls_4'] = zeros

In [14]:
from sklearn.model_selection import KFold
kf = KFold(n_splits = 5)
df = df4.reset_index(drop=True)
df['fold'] = -1

for fold, (train_idx, val_idx) in enumerate(kf.split(df, y=areas_1000)):
    df.loc[val_idx, 'fold'] = fold
    
display(df.fold.value_counts())   
df.head(5)

0    2380
1    2380
2    2380
3    2379
4    2379
Name: fold, dtype: int64

,alls_4,path_4,nonpass4,path,nonpath,nonalls_4,fold
0,"{'x': 256, 'y': 144, 'width': 100, 'height': 64}",./img512_288/cots_crops/cotscrop-video_0-16-0.jpg,./img512_288/notcots_crops/notcotscrop-video_2...,cots_crops/cotscrop-video_0-16-0.jpg,notcots_crops/notcotscrop-video_2-4331-0.jpg,0,0
1,"{'x': 256, 'y': 144, 'width': 100, 'height': 64}",./img512_288/cots_crops/cotscrop-video_0-17-0.jpg,./img512_288/notcots_crops/notcotscrop-video_1...,cots_crops/cotscrop-video_0-17-0.jpg,notcots_crops/notcotscrop-video_1-8615-0.jpg,0,0
2,"{'x': 256, 'y': 144, 'width': 100, 'height': 64}",./img512_288/cots_crops/cotscrop-video_0-18-0.jpg,./img512_288/notcots_crops/notcotscrop-video_2...,cots_crops/cotscrop-video_0-18-0.jpg,notcots_crops/notcotscrop-video_2-513-0.jpg,0,0
3,"{'x': 256, 'y': 144, 'width': 100, 'height': 64}",./img512_288/cots_crops/cotscrop-video_0-19-0.jpg,./img512_288/notcots_crops/notcotscrop-video_0...,cots_crops/cotscrop-video_0-19-0.jpg,notcots_crops/notcotscrop-video_0-2020-0.jpg,0,0
4,"{'x': 256, 'y': 144, 'width': 100, 'height': 64}",./img512_288/cots_crops/cotscrop-video_0-20-0.jpg,./img512_288/notcots_crops/notcotscrop-video_1...,cots_crops/cotscrop-video_0-20-0.jpg,notcots_crops/notcotscrop-video_1-819-0.jpg,0,0


In [15]:
df[df['fold']==1]

,alls_4,path_4,nonpass4,path,nonpath,nonalls_4,fold
2380,"{'x': 256, 'y': 144, 'width': 128, 'height': 122}",./img512_288/cots_crops/cotscrop-video_0-9678-...,./img512_288/notcots_crops/notcotscrop-video_2...,cots_crops/cotscrop-video_0-9678-0.jpg,notcots_crops/notcotscrop-video_2-2513-0.jpg,0,1
2381,"{'x': 256, 'y': 144, 'width': 88, 'height': 86}",./img512_288/cots_crops/cotscrop-video_0-9678-...,./img512_288/notcots_crops/notcotscrop-video_2...,cots_crops/cotscrop-video_0-9678-1.jpg,notcots_crops/notcotscrop-video_2-2513-1.jpg,0,1
2382,"{'x': 256, 'y': 144, 'width': 98, 'height': 102}",./img512_288/cots_crops/cotscrop-video_0-9678-...,./img512_288/notcots_crops/notcotscrop-video_2...,cots_crops/cotscrop-video_0-9678-2.jpg,notcots_crops/notcotscrop-video_2-2513-2.jpg,0,1
2383,"{'x': 256, 'y': 144, 'width': 70, 'height': 66}",./img512_288/cots_crops/cotscrop-video_0-9678-...,./img512_288/notcots_crops/notcotscrop-video_2...,cots_crops/cotscrop-video_0-9678-3.jpg,notcots_crops/notcotscrop-video_2-2513-3.jpg,0,1
2384,"{'x': 256, 'y': 144, 'width': 130, 'height': 126}",./img512_288/cots_crops/cotscrop-video_0-9679-...,./img512_288/notcots_crops/notcotscrop-video_2...,cots_crops/cotscrop-video_0-9679-0.jpg,notcots_crops/notcotscrop-video_2-5563-0.jpg,0,1
...,...,...,...,...,...,...,...
4755,"{'x': 256, 'y': 144, 'width': 106, 'height': 96}",./img512_288/cots_crops/cotscrop-video_1-4756-...,./img512_288/notcots_crops/notcotscrop-video_2...,cots_crops/cotscrop-video_1-4756-0.jpg,notcots_crops/notcotscrop-video_2-10344-0.jpg,0,1
4756,"{'x': 256, 'y': 144, 'width': 104, 'height': 92}",./img512_288/cots_crops/cotscrop-video_1-4757-...,./img512_288/notcots_crops/notcotscrop-video_1...,cots_crops/cotscrop-video_1-4757-0.jpg,notcots_crops/notcotscrop-video_1-2143-0.jpg,0,1
4757,"{'x': 256, 'y': 144, 'width': 96, 'height': 88}",./img512_288/cots_crops/cotscrop-video_1-4758-...,./img512_288/notcots_crops/notcotscrop-video_0...,cots_crops/cotscrop-video_1-4758-0.jpg,notcots_crops/notcotscrop-video_0-6055-0.jpg,0,1
4758,"{'x': 256, 'y': 144, 'width': 86, 'height': 80}",./img512_288/cots_crops/cotscrop-video_1-4759-...,./img512_288/notcots_crops/notcotscrop-video_0...,cots_crops/cotscrop-video_1-4759-0.jpg,notcots_crops/notcotscrop-video_0-5852-0.jpg,0,1


In [16]:
df.shape

(11898, 7)

In [17]:
!mkdir -p ./yolo_data/fold2/images/val
!mkdir -p ./yolo_data/fold2/images/train

!mkdir -p ./yolo_data/fold2/labels/val
!mkdir -p ./yolo_data/fold2/labels/train

In [18]:
!ls

__notebook__.ipynb  yolo_data


'''
fold = 1
annos = []
for i, x in train.iterrows():
    if x.video_id == fold:
        mode = 'val'
    else:
        # train
        mode = 'train'
        if not x.pos: continue
        # val
    copyfile(f'../input/tensorflow-great-barrier-reef/train_images/video_{x.video_id}/{x.video_frame}.jpg',
                f'./yolo_data/fold{fold}/images/{mode}/{x.image_id}.jpg')
    if not x.pos:
        continue
    r = ''
    anno = eval(x.annotations)
    for an in anno:
#            annos.append(an)
        r += '0 {} {} {} {}\n'.format((an['x'] + an['width'] / 2) / 1280,
                                        (an['y'] + an['height'] / 2) / 720,
                                        an['width'] / 1280, an['height'] / 720)
    with open(f'./yolo_data/fold{fold}/labels/{mode}/{x.image_id}.txt', 'w') as fp:
        fp.write(r)

'''

In [19]:
fold = 2
z=0
annos = []
for i, x in df.iterrows():
    if x.fold == fold:
        mode = 'val'
    else:
        # train
        mode = 'train'
        #if not x.pos: continue
        # val
    copyfile(f'../input/cots-cropimg-04-512-288/{x.path}',
                f'./yolo_data/fold{fold}/images/{mode}/crop_{z}.jpg')
    #if not x.pos:
    #    continue
    r = ''
    anno = ast.literal_eval(x.alls_4)
    #for an in anno:
    an = anno
#            annos.append(an)
    r += '0 {} {} {} {}\n'.format((an['x'] + an['width'] / 2) / 512,
                                    (an['y'] + an['height'] / 2) / 288,
                                    an['width'] / 512, an['height'] / 288)
    with open(f'./yolo_data/fold{fold}/labels/{mode}/crop_{z}.txt', 'w') as fp:
        fp.write(r)
    z+=1

In [20]:
hyps = '''
# YOLOv5 by Ultralytics, GPL-3.0 license
# Hyperparameters for COCO training from scratch
# python train.py --batch 40 --cfg yolov5m.yaml --weights '' --data coco.yaml --img 640 --epochs 300
# See tutorials for hyperparameter evolution https://github.com/ultralytics/yolov5#tutorials

lr0: 0.001  # initial learning rate (SGD=1E-2, Adam=1E-3)
lrf: 0.1  # final OneCycleLR learning rate (lr0 * lrf)
momentum: 0.937  # SGD momentum/Adam beta1
beta: 1
weight_decay: 0.0005  # optimizer weight decay 5e-4
warmup_epochs: 3.0  # warmup epochs (fractions ok)
warmup_momentum: 0.8  # warmup initial momentum
warmup_bias_lr: 0.1  # warmup initial bias lr
box: 0.05  # box loss gain
cls: 0.5  # cls loss gain
cls_pw: 1.0  # cls BCELoss positive_weight
obj: 1.0  # obj loss gain (scale with pixels)
obj_pw: 1.0  # obj BCELoss positive_weight
iou_t: 0.20  # IoU training threshold
anchor_t: 4.0  # anchor-multiple threshold
# anchors: 3  # anchors per output layer (0 to ignore)
fl_gamma: 0.0  # focal loss gamma (efficientDet default gamma=1.5)
hsv_h: 0.015  # image HSV-Hue augmentation (fraction)
hsv_s: 0.7  # image HSV-Saturation augmentation (fraction)
hsv_v: 0.4  # image HSV-Value augmentation (fraction)
degrees: 0.0  # image rotation (+/- deg)
translate: 0.1  # image translation (+/- fraction)
scale: 0.5  # image scale (+/- gain)
shear: 0.0  # image shear (+/- deg)
perspective: 0.0  # image perspective (+/- fraction), range 0-0.001
flipud: 0.5  # image flip up-down (probability)
fliplr: 0.5  # image flip left-right (probability)
mosaic: 1.0  # image mosaic (probability)
mixup: 0.5  # image mixup (probability)
copy_paste: 0.0  # segment copy-paste (probability)
'''

In [21]:
data = '''
# Train/val/test sets as 1) dir: path/to/imgs, 2) file: path/to/imgs.txt, or 3) list: [path/to/imgs1, path/to/imgs2, ..]
path: ../yolo_data/fold2/  # dataset root dir
train: images/train  # train images (relative to 'path') 128 images
val: images/val  # val images (relative to 'path') 128 images
test:  # test images (optional)

# Classes
nc: 1  # number of classes
names: ['reef']  # class names


# Download script/URL (optional)
# download: https://ultralytics.com/assets/coco128.zip
'''

In [22]:
!git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...
remote: Enumerating objects: 10866, done.
remote: Total 10866 (delta 0), reused 0 (delta 0), pack-reused 10866
Receiving objects: 100% (10866/10866), 11.01 MiB | 13.80 MiB/s, done.
Resolving deltas: 100% (7503/7503), done.


In [23]:
with open('./yolov5/data/reef_f1_naive.yaml', 'w') as fp:
    fp.write(data)
with open('./yolov5/data/hyps/hyp.heavy.2.yaml', 'w') as fp:
    fp.write(hyps)

In [24]:
%cd yolov5

/kaggle/working/yolov5


In [25]:
!ls data/

Argoverse.yaml	      VOC.yaml	     hyps		 xView.yaml
GlobalWheat2020.yaml  VisDrone.yaml  images
Objects365.yaml       coco.yaml      reef_f1_naive.yaml
SKU-110K.yaml	      coco128.yaml   scripts


In [26]:
!python train.py --img 512 --batch 16 --epochs 20 --multi-scale --optimizer Adam --save-period 1 --patience 5 --data reef_f1_naive.yaml --weights yolov5s6.pt --name s6_512_batch16_10epochs --hyp data/hyps/hyp.heavy.2.yaml

wandb: Currently logged in as: mochi22 (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.10 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.12.7
wandb: Syncing run s6_512_batch16_10epochs
wandb: ⭐️ View project at https://wandb.ai/mochi22/YOLOv5
wandb: 🚀 View run at https://wandb.ai/mochi22/YOLOv5/runs/2w6cgoms
wandb: Run data is saved locally in /kaggle/working/yolov5/wandb/run-20220214_081245-2w6cgoms
wandb: Run `wandb offline` to turn off syncing.
















































wandb: Waiting for W&B process to finish, PID 154... (success).
wandb:                                                                                
wandb: Run history:
wandb:        metrics/mAP_0.5 ▁▁▁▁▃▂▃▄▄▆▃▅▄▆▆▅▅██▇█
wandb:   metrics/mAP_0.5:0.95 ▁▁▁▁▂▂▂▄▃▅▃▅▃▅▆▅▅██▇█
wandb:      metrics/precision ▁▁▁▁▃▂▃▅▄▆▄▅▅▆▆▅▆▇█▇█
wandb:         metrics/recall ▇▁▆▃▂▃▃▅▆▇▆█▅▆▇▇▆█▇▇▇
wandb:         train/box_

In [27]:
!ls

CONTRIBUTING.md  __pycache__  hubconf.py	setup.cfg	val.py
Dockerfile	 data	      models		train.py	wandb
LICENSE		 detect.py    requirements.txt	tutorial.ipynb	yolov5s6.pt
README.md	 export.py    runs		utils
